<a href="https://colab.research.google.com/github/y-arjun-y/liars-miller-rabin/blob/main/liars_miller_rabin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Liars in the Miller-Rabin primality test by Arjun Yadav
## All of the code and results are available on [GitHub](https://github.com/y-arjun-y/liars-miller-rabin). Inspired by [Numberphile](https://www.youtube.com/watch?v=_MscGSN5J6o).

## Implementation of the Miller-Rabin primality test in Python

In [1]:
'''
Function by https://rosettacode.org/wiki/Miller%E2%80%93Rabin_primality_test#Python with slight changes by me to support a custom witness "a". 
Function and its formula is based on https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test.
A return value of false means n in not prime (i.e. composite).
A return value of true means n is very likely to be a prime (likelihood depending on the number of rounds).
'''

def miller_rabin_base(n, a):
  # checking if it is an integer
    if n != int(n):
        return False

    n = int(n)

    # base cases
    if n == 0 or n == 1 or n == 4 or n == 6 or n == 8 or n == 9:
        return False

    if n == 2 or n == 3 or n == 5 or n == 7:
        return True

    if n % 2 == 0:
        return False

    s = 0
    d = n - 1

    while d % 2 == 0:
        d >>= 1
        s += 1
    
    assert(2**s * d == n - 1)

    # trial run
    def trial_composite(a):
        if pow(a, d, n) == 1:
            return False

        for i in range(s):
            if pow(a, 2**i * d, n) == n - 1:
                return False
        return True

    # number of trials rounds
    num_trials = 10

    for _ in range(num_trials):
        if trial_composite(a):
            return False

    return True

## Supporting multiple witnesses in the Miller-Rabin primality test

In [2]:
'''
Returns false if any one of the witness returns false and returns true if all witnesses return true.
'''

def miller_rabin(num, witness_list):
  result = []
  
  for i in range(len(witness_list)):
    result.append(miller_rabin_base(num, witness_list[i]))
  
  if False in result:
    return False
  
  return True

## Simple prime number test

In [3]:
'''
Returns whether a number is prime (true) or not (false).
Function by https://en.wikipedia.org/wiki/Primality_test#Python.
'''

def actual_prime(n):
    if n <= 3:
        return n > 1
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i ** 2 <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

## Finding the worst witness in the Miller-Rabin primality test (for `n` in a certain range, 1 witness)

In [32]:
from tqdm import tqdm

DEFENDANT_RANGE = 5000 # change me!
DEFENDANTS = [i for i in range(DEFENDANT_RANGE + 1) if i == 2 or i % 2 != 0]
if DEFENDANT_RANGE % 2 == 0:
  WITNESS_RANGE = DEFENDANT_RANGE - 1
else:
  WITNESS_RANGE = DEFENDANT_RANGE

results = {}

# trial loop function
def trial_loop(defendants, witness):
  lie_count = 0

  for defendant in defendants:
      if miller_rabin_base(defendant, witness) != actual_prime(defendant):
        lie_count += 1
  
  return lie_count

for i in tqdm(range(WITNESS_RANGE)):
  results[i] = trial_loop(DEFENDANTS, i)

# finding top 100 liars
for i in sorted(results.items(), key=lambda x: x[1], reverse=True)[:100]:
  print(i)

100%|██████████| 4999/4999 [02:47<00:00, 29.91it/s]

(1, 1830)
(0, 665)
(4096, 70)
(256, 39)
(4624, 36)
(4724, 36)
(1432, 33)
(2474, 33)
(4117, 33)
(4952, 33)
(1296, 32)
(1451, 32)
(1918, 32)
(4562, 32)
(3464, 31)
(4124, 31)
(4852, 31)
(4951, 31)
(2414, 30)
(3194, 30)
(3449, 30)
(4726, 30)
(4913, 30)
(1322, 29)
(1576, 29)
(1607, 29)
(2393, 29)
(2582, 29)
(2834, 29)
(3446, 29)
(3466, 29)
(4094, 29)
(4274, 29)
(4454, 29)
(1303, 28)
(1816, 28)
(1844, 28)
(2116, 28)
(2789, 28)
(2971, 28)
(3149, 28)
(3376, 28)
(3536, 28)
(3674, 28)
(4330, 28)
(901, 27)
(1156, 27)
(1207, 27)
(1979, 27)
(2382, 27)
(2729, 27)
(2792, 27)
(3044, 27)
(3106, 27)
(3136, 27)
(3254, 27)
(3314, 27)
(3571, 27)
(3656, 27)
(3926, 27)
(4079, 27)
(4086, 27)
(4232, 27)
(4276, 27)
(4346, 27)
(4558, 27)
(4747, 27)
(4924, 27)
(16, 26)
(374, 26)
(676, 26)
(900, 26)
(1174, 26)
(1832, 26)
(1889, 26)
(2078, 26)
(2174, 26)
(2458, 26)
(2536, 26)
(2806, 26)
(3526, 26)
(3781, 26)
(3826, 26)
(4164, 26)
(4589, 26)
(4713, 26)
(4749, 26)
(4784, 26)
(4934, 26)
(874, 25)
(1101, 25)
(1568, 25)

## Finding the worst witnesses in the Miller-Rabin primality test (for `n` in a certain range, 2 witnesses)

In [33]:
from tqdm import tqdm
import itertools

DEFENDANT_RANGE = 300 # change me!
DEFENDANTS = [i for i in range(DEFENDANT_RANGE + 1) if i == 2 or i % 2 != 0]
if DEFENDANT_RANGE % 2 == 0:
  WITNESS_RANGE = DEFENDANT_RANGE - 1
else:
  WITNESS_RANGE = DEFENDANT_RANGE
WITNESSES = list(itertools.product([i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)]))

results = {}

# trial loop function
def trial_loop(defendants, witnesses):
  lie_count = 0

  for defendant in defendants:
      if miller_rabin(defendant, [witnesses[0], witnesses[1]]) != actual_prime(defendant):
        lie_count += 1
  
  return lie_count

for i in tqdm(WITNESSES):
  results[i] = trial_loop(DEFENDANTS, (i))

# finding top 100 liars
for i in sorted(results.items(), key=lambda x: x[1], reverse=True)[:500]:
  print(i)

100%|██████████| 89401/89401 [06:00<00:00, 248.04it/s]

((1, 1), 87)
((0, 0), 58)
((0, 1), 58)
((0, 2), 58)
((0, 3), 58)
((0, 4), 58)
((0, 5), 58)
((0, 6), 58)
((0, 7), 58)
((0, 8), 58)
((0, 9), 58)
((0, 10), 58)
((0, 11), 58)
((0, 12), 58)
((0, 13), 58)
((0, 14), 58)
((0, 15), 58)
((0, 16), 58)
((0, 17), 58)
((0, 18), 58)
((0, 19), 58)
((0, 20), 58)
((0, 21), 58)
((0, 22), 58)
((0, 23), 58)
((0, 24), 58)
((0, 25), 58)
((0, 26), 58)
((0, 27), 58)
((0, 28), 58)
((0, 29), 58)
((0, 30), 58)
((0, 31), 58)
((0, 32), 58)
((0, 33), 58)
((0, 34), 58)
((0, 35), 58)
((0, 36), 58)
((0, 37), 58)
((0, 38), 58)
((0, 39), 58)
((0, 40), 58)
((0, 41), 58)
((0, 42), 58)
((0, 43), 58)
((0, 44), 58)
((0, 45), 58)
((0, 46), 58)
((0, 47), 58)
((0, 48), 58)
((0, 49), 58)
((0, 50), 58)
((0, 51), 58)
((0, 52), 58)
((0, 53), 58)
((0, 54), 58)
((0, 55), 58)
((0, 56), 58)
((0, 57), 58)
((0, 58), 58)
((0, 59), 58)
((0, 60), 58)
((0, 61), 58)
((0, 62), 58)
((0, 63), 58)
((0, 64), 58)
((0, 65), 58)
((0, 66), 58)
((0, 67), 58)
((0, 68), 58)
((0, 69), 58)
((0, 70), 58)
((0

## Finding the worst witnesses in the Miller-Rabin primality test (for `n` in a certain range, 3 witnesses)

In [26]:
from tqdm import tqdm
import itertools

DEFENDANT_RANGE = 75 # change me!
DEFENDANTS = [i for i in range(DEFENDANT_RANGE + 1) if i == 2 or i % 2 != 0]
if DEFENDANT_RANGE % 2 == 0:
  WITNESS_RANGE = DEFENDANT_RANGE - 1
else:
  WITNESS_RANGE = DEFENDANT_RANGE
WITNESSES = list(itertools.product([i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)]))

results = {}

# trial loop function
def trial_loop(defendants, witnesses):
  lie_count = 0

  for defendant in defendants:
      if miller_rabin(defendant, [witnesses[0], witnesses[1]]) != actual_prime(defendant):
        lie_count += 1
  
  return lie_count

for i in tqdm(WITNESSES):
  results[i] = trial_loop(DEFENDANTS, (i))

# finding top 100 liars
for i in sorted(results.items(), key=lambda x: x[1], reverse=True)[:500]:
  print(i)

100%|██████████| 421875/421875 [06:43<00:00, 1044.50it/s]


((0, 0, 0), 17)
((0, 0, 1), 17)
((0, 0, 2), 17)
((0, 0, 3), 17)
((0, 0, 4), 17)
((0, 0, 5), 17)
((0, 0, 6), 17)
((0, 0, 7), 17)
((0, 0, 8), 17)
((0, 0, 9), 17)
((0, 0, 10), 17)
((0, 0, 11), 17)
((0, 0, 12), 17)
((0, 0, 13), 17)
((0, 0, 14), 17)
((0, 0, 15), 17)
((0, 0, 16), 17)
((0, 0, 17), 17)
((0, 0, 18), 17)
((0, 0, 19), 17)
((0, 0, 20), 17)
((0, 0, 21), 17)
((0, 0, 22), 17)
((0, 0, 23), 17)
((0, 0, 24), 17)
((0, 0, 25), 17)
((0, 0, 26), 17)
((0, 0, 27), 17)
((0, 0, 28), 17)
((0, 0, 29), 17)
((0, 0, 30), 17)
((0, 0, 31), 17)
((0, 0, 32), 17)
((0, 0, 33), 17)
((0, 0, 34), 17)
((0, 0, 35), 17)
((0, 0, 36), 17)
((0, 0, 37), 17)
((0, 0, 38), 17)
((0, 0, 39), 17)
((0, 0, 40), 17)
((0, 0, 41), 17)
((0, 0, 42), 17)
((0, 0, 43), 17)
((0, 0, 44), 17)
((0, 0, 45), 17)
((0, 0, 46), 17)
((0, 0, 47), 17)
((0, 0, 48), 17)
((0, 0, 49), 17)
((0, 0, 50), 17)
((0, 0, 51), 17)
((0, 0, 52), 17)
((0, 0, 53), 17)
((0, 0, 54), 17)
((0, 0, 55), 17)
((0, 0, 56), 17)
((0, 0, 57), 17)
((0, 0, 58), 17)
((0, 0,

## Finding the worst witnesses in the Miller-Rabin primality test (for `n` in a certain range, 4 witnesses)

In [30]:
from tqdm import tqdm
import itertools

DEFENDANT_RANGE = 25 # change me!
DEFENDANTS = [i for i in range(DEFENDANT_RANGE + 1) if i == 2 or i % 2 != 0]
if DEFENDANT_RANGE % 2 == 0:
  WITNESS_RANGE = DEFENDANT_RANGE - 1
else:
  WITNESS_RANGE = DEFENDANT_RANGE
WITNESSES = list(itertools.product([i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)]))

results = {}

# trial loop function
def trial_loop(defendants, witnesses):
  lie_count = 0

  for defendant in defendants:
      if miller_rabin(defendant, [witnesses[0], witnesses[1]]) != actual_prime(defendant):
        lie_count += 1
  
  return lie_count

for i in tqdm(WITNESSES):
  results[i] = trial_loop(DEFENDANTS, (i))

# finding top 100 liars
for i in sorted(results.items(), key=lambda x: x[1], reverse=True)[:500]:
  print(i)

100%|██████████| 390625/390625 [01:44<00:00, 3756.00it/s]


((0, 0, 0, 0), 5)
((0, 0, 0, 1), 5)
((0, 0, 0, 2), 5)
((0, 0, 0, 3), 5)
((0, 0, 0, 4), 5)
((0, 0, 0, 5), 5)
((0, 0, 0, 6), 5)
((0, 0, 0, 7), 5)
((0, 0, 0, 8), 5)
((0, 0, 0, 9), 5)
((0, 0, 0, 10), 5)
((0, 0, 0, 11), 5)
((0, 0, 0, 12), 5)
((0, 0, 0, 13), 5)
((0, 0, 0, 14), 5)
((0, 0, 0, 15), 5)
((0, 0, 0, 16), 5)
((0, 0, 0, 17), 5)
((0, 0, 0, 18), 5)
((0, 0, 0, 19), 5)
((0, 0, 0, 20), 5)
((0, 0, 0, 21), 5)
((0, 0, 0, 22), 5)
((0, 0, 0, 23), 5)
((0, 0, 0, 24), 5)
((0, 0, 1, 0), 5)
((0, 0, 1, 1), 5)
((0, 0, 1, 2), 5)
((0, 0, 1, 3), 5)
((0, 0, 1, 4), 5)
((0, 0, 1, 5), 5)
((0, 0, 1, 6), 5)
((0, 0, 1, 7), 5)
((0, 0, 1, 8), 5)
((0, 0, 1, 9), 5)
((0, 0, 1, 10), 5)
((0, 0, 1, 11), 5)
((0, 0, 1, 12), 5)
((0, 0, 1, 13), 5)
((0, 0, 1, 14), 5)
((0, 0, 1, 15), 5)
((0, 0, 1, 16), 5)
((0, 0, 1, 17), 5)
((0, 0, 1, 18), 5)
((0, 0, 1, 19), 5)
((0, 0, 1, 20), 5)
((0, 0, 1, 21), 5)
((0, 0, 1, 22), 5)
((0, 0, 1, 23), 5)
((0, 0, 1, 24), 5)
((0, 0, 2, 0), 5)
((0, 0, 2, 1), 5)
((0, 0, 2, 2), 5)
((0, 0, 2, 3), 5